In [1]:
import tkinter as tk
import random
import time
from queue import Queue, PriorityQueue
from tkinter import messagebox  # For showing warning messages

CANVAS_SIZE = 600
TILE_SIZE = 100  # Size of each tile in the puzzle

# Goal state of the 8-puzzle
PUZZLE_GOAL_STATE = [1, 2, 3, 4, 5, 6, 7, 8, 0]  # 0 represents the empty tile

class PuzzleApp:
    def __init__(self, root):
        self.root = root
        self.root.title("8-Puzzle Solver")

        # Create canvas for displaying the puzzle
        self.canvas = tk.Canvas(self.root, width=CANVAS_SIZE, height=CANVAS_SIZE, bg='white')
        self.canvas.grid(row=0, column=0, columnspan=4)

        # Control buttons for each algorithm
        self.bfs_button = tk.Button(self.root, text="Solve with BFS", command=lambda: self.solve_puzzle(bfs_puzzle, "BFS"))
        self.bfs_button.grid(row=1, column=0)

        self.dfs_button = tk.Button(self.root, text="Solve with DFS", command=self.warn_and_solve_dfs)
        self.dfs_button.grid(row=1, column=1)

        self.astar_button = tk.Button(self.root, text="Solve with A*", command=lambda: self.solve_puzzle(astar_puzzle, "A*"))
        self.astar_button.grid(row=1, column=2)

        self.bidirectional_button = tk.Button(self.root, text="Bidirectional Search", command=lambda: self.solve_puzzle(bidirectional_search_puzzle, "Bidirectional"))
        self.bidirectional_button.grid(row=1, column=3)

        self.random_button = tk.Button(self.root, text="Randomize Puzzle", command=self.randomize_puzzle)
        self.random_button.grid(row=2, column=1, columnspan=2)

        # Initialize puzzle state
        self.puzzle = PUZZLE_GOAL_STATE[:]
        self.solution_path = []

        self.display_puzzle()

    def warn_and_solve_dfs(self):
        """Warn the user before running DFS, as it can take a long time."""
        response = messagebox.askyesno("Warning", "DFS can take a lot of time to find the solution. Do you want to proceed?")
        if response:
            self.solve_puzzle(dfs_puzzle, "DFS")

    def randomize_puzzle(self):
        """Randomizes the puzzle tiles to create a solvable puzzle."""
        self.puzzle = PUZZLE_GOAL_STATE[:]
        random.shuffle(self.puzzle)
        while not self.is_solvable(self.puzzle):
            random.shuffle(self.puzzle)
        self.display_puzzle()

    def display_puzzle(self):
        """Display the current puzzle state on the canvas."""
        self.canvas.delete("all")
        size = int(len(self.puzzle) ** 0.5)

        for i in range(size):
            for j in range(size):
                value = self.puzzle[i * size + j]
                if value != 0:
                    x1, y1 = j * TILE_SIZE, i * TILE_SIZE
                    x2, y2 = x1 + TILE_SIZE, y1 + TILE_SIZE
                    self.canvas.create_rectangle(x1, y1, x2, y2, fill="lightblue", outline="black")
                    self.canvas.create_text((x1 + x2) / 2, (y1 + y2) / 2, text=str(value), font=("Arial", 24))

    def solve_puzzle(self, algorithm, name):
        """Solve the puzzle using the selected algorithm."""
        start_time = time.perf_counter()
        solution_path = algorithm(self.puzzle, PUZZLE_GOAL_STATE)
        elapsed_time = time.perf_counter() - start_time

        if solution_path:
            print(f"{name} Solution Found! Time: {elapsed_time:.6f}s")
            self.animate_solution(solution_path)
        else:
            print(f"{name} could not find a solution.")

    def animate_solution(self, solution_path):
        """Animate the solution path on the canvas."""
        for state in solution_path:
            self.puzzle = state
            self.display_puzzle()
            self.root.update()
            time.sleep(1)  # Add delay for smooth animation

    @staticmethod
    def is_solvable(puzzle):
        """Check if a puzzle is solvable."""
        inv_count = 0
        flattened = [num for num in puzzle if num != 0]
        for i in range(len(flattened) - 1):
            for j in range(i + 1, len(flattened)):
                if flattened[i] > flattened[j]:
                    inv_count += 1
        return inv_count % 2 == 0

# BFS Algorithm for 8-Puzzle
def bfs_puzzle(start, goal):
    queue = Queue()
    queue.put([start])
    visited = set()
    visited.add(tuple(start))

    while not queue.empty():
        path = queue.get()
        state = path[-1]

        if state == goal:
            return path

        for next_state in generate_successors(state):
            if tuple(next_state) not in visited:
                visited.add(tuple(next_state))
                queue.put(path + [next_state])

    return None

# DFS Algorithm for 8-Puzzle
def dfs_puzzle(start, goal):
    stack = [[start]]
    visited = set()
    visited.add(tuple(start))

    while stack:
        path = stack.pop()
        state = path[-1]

        if state == goal:
            return path

        for next_state in generate_successors(state):
            if tuple(next_state) not in visited:
                visited.add(tuple(next_state))
                stack.append(path + [next_state])

    return None

# A* Algorithm for 8-Puzzle
def astar_puzzle(start, goal):
    def heuristic(state):
        """Heuristic: Sum of Manhattan distances of tiles from their goal positions."""
        size = int(len(state) ** 0.5)
        total_distance = 0
        for i, value in enumerate(state):
            if value != 0:
                target_row, target_col = divmod(goal.index(value), size)
                current_row, current_col = divmod(i, size)
                total_distance += abs(target_row - current_row) + abs(target_col - current_col)
        return total_distance

    pq = PriorityQueue()
    pq.put((0, [start]))
    visited = set()
    visited.add(tuple(start))

    while not pq.empty():
        _, path = pq.get()
        state = path[-1]

        if state == goal:
            return path

        for next_state in generate_successors(state):
            if tuple(next_state) not in visited:
                visited.add(tuple(next_state))
                new_path = path + [next_state]
                cost = len(new_path) + heuristic(next_state)
                pq.put((cost, new_path))

    return None

# Bidirectional Search for 8-Puzzle
def bidirectional_search_puzzle(start, goal):
    if start == goal:
        return [start]

    front_queue = Queue()
    back_queue = Queue()

    front_queue.put([start])
    back_queue.put([goal])

    front_visited = {tuple(start): [start]}
    back_visited = {tuple(goal): [goal]}

    while not front_queue.empty() and not back_queue.empty():
        front_path = front_queue.get()
        front_state = front_path[-1]

        back_path = back_queue.get()
        back_state = back_path[-1]

        if tuple(front_state) in back_visited:
            return front_path + back_visited[tuple(front_state)][::-1][1:]

        if tuple(back_state) in front_visited:
            return front_visited[tuple(back_state)] + back_path[::-1][1:]

        for next_state in generate_successors(front_state):
            if tuple(next_state) not in front_visited:
                front_visited[tuple(next_state)] = front_path + [next_state]
                front_queue.put(front_path + [next_state])

        for next_state in generate_successors(back_state):
            if tuple(next_state) not in back_visited:
                back_visited[tuple(next_state)] = back_path + [next_state]
                back_queue.put(back_path + [next_state])

    return None

# Helper function to generate successor states
def generate_successors(state):
    """Generate all valid successors by sliding tiles."""
    size = int(len(state) ** 0.5)
    zero_index = state.index(0)
    row, col = divmod(zero_index, size)

    successors = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    for dr, dc in directions:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < size and 0 <= new_col < size:
            new_index = new_row * size + new_col
            new_state = state[:]
            new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
            successors.append(new_state)

    return successors

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = PuzzleApp(root)
    root.mainloop()


BFS Solution Found! Time: 0.273237s
